In [0]:
# Step 1: Install and import dependencies
!pip install openai --upgrade

In [0]:
import openai
import pandas as pd

In [0]:

# Step 3: Campaign Dictionary
campaign_dict = {
    "loyalty_reward": {
        "campaign_type": "Loyalty Reward",
        "offer_template": "Exclusive reward: $20 off your next {top_category} purchase",
        "tone": "Warm and appreciative",
        "call_to_action": "Claim Your Reward"
    },
    "reactivation": {
        "campaign_type": "We Miss You",
        "offer_template": "Here’s 25% off to welcome you back to our {top_category} collection!",
        "tone": "Friendly and encouraging",
        "call_to_action": "Come Back Today"
    },
    "seasonal_sale": {
        "campaign_type": "Seasonal Sale",
        "offer_template": "Seasonal offer: 30% off all {top_category} items!",
        "tone": "Exciting and persuasive",
        "call_to_action": "Shop the Sale"
    },
    "new_arrival": {
        "campaign_type": "New Arrival",
        "offer_template": "Discover the latest in {top_category} — just arrived!",
        "tone": "Fresh and trendy",
        "call_to_action": "Explore Now"
    }
}

# Step 4: Define Prompt Generator
def build_prompt(row):
    template = campaign_dict[row["campaign_key"]]
    offer = template["offer_template"].format(top_category=row["top_category"])

    return f"""Write a personalized marketing email.

Persona: {row['persona_sentence']}
Segment: {row['persona_cluster_label']}
Campaign Type: {template['campaign_type']}
Offer: “{offer}”
Tone: {template['tone']}
Call-to-action: “{template['call_to_action']}”
"""

# Step 5: Load sample guests from Spark
spark_df = spark.table("bits_pilani.gold_sch.guest_persona_campaign_enriched")
sample_df = spark_df.limit(10).toPandas()

# Step 6: Generate prompts
sample_df["prompt"] = sample_df.apply(build_prompt, axis=1)
display(sample_df)

In [0]:
# Step 0: Import and initialize OpenAI client (v1+)
from openai import OpenAI

# 2. Retrieve the secret in Python
# openai_api_key = dbutils.secrets.get(scope="openai", key="openai_api_key")

client = OpenAI
(api_key="")  # Replace with your key or use environment variable



# Step 7: Generate email content via OpenAI
def generate_email(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a marketing assistant helping write email content."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"ERROR: {e}"

# Apply to DataFrame
sample_df["email_generated"] = sample_df["prompt"].apply(generate_email)


In [0]:
display(sample_df)

In [0]:
# Step 8: Save or export results
sample_df[[
    "MASTER_GUEST_ID", "persona_cluster_label", "campaign_key","prompt","cluster_sentence","persona_sentence", "email_generated"
]].to_csv("generated_emails_sample.csv", index=False)

# Optional: Save to Spark table
df_result = spark.createDataFrame(sample_df)
df_result.write.mode("overwrite").saveAsTable("bits_pilani.gold_sch.generated_emails_sample")